In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# 1. 读取Excel文件，查看数据基本信息
# 读取弹幕数据
df = pd.read_excel('D:/CITYU COURSES/5507/homework/弹幕/BV17JRKYdEEs弹幕数据.xlsx')

# 查看数据基本信息
print("="*50)
print("原始数据基本信息")
print("="*50)
print(f"数据形状: {df.shape} (行数, 列数)")
print(f"\n列名列表: {list(df.columns)}")
print(f"\n前5行数据:")
print(df.head())
print(f"\n数据类型:")
print(df.dtypes)
print(f"\n缺失值统计:")
print(df.isnull().sum())

原始数据基本信息
数据形状: (872, 1) (行数, 列数)

列名列表: ['内容']

前5行数据:
                  内容
0  “直行或是转弯，我们最终都会相见”
1    人可以干干净净的走，它们也可以
2       别说参与了就这么看都泪目
3         乌龟：我的续航非常久
4        画面色彩富士味儿很足啊

数据类型:
内容    object
dtype: object

缺失值统计:
内容    0
dtype: int64


In [1]:
# 宠物殡葬B站弹幕数据清洗最终版
# 修复：PermissionError权限问题 + NameError变量问题 + 主题过滤
import pandas as pd
import re
import os
import time
import warnings
warnings.filterwarnings('ignore')

# ===================== 核心配置：主题关键词（可自定义调整） =====================
# 宠物殡葬相关关键词（覆盖宠物类型、殡葬服务、情感表达）
PET_FUNERAL_KEYWORDS = {
    # 宠物类型
    '猫', '喵', '猫咪', '小猫', '猫主子', '狗', '汪', '狗狗', '小狗', '宠物',
    '毛孩子', '主子', '小家伙', '小动物', '龟', '乌龟', '仓鼠', '兔子', '龙猫',
    # 殡葬相关
    '殡葬', '葬礼', '安葬', '火化', '告别', '送行', '安息', '长眠', '离世', '去世',
    '逝去', '离开', '走了', '往生', '归西', '后事', '告别仪式', '骨灰', '墓碑',
    # 核心情感
    '泪目', '哭了', '难过', '心疼', '感动', '温暖', '治愈', '安心', '不舍',
    '怀念', '思念', '想念', '尊重', '体面', '尊严', '好好告别',
    # 相关服务/场景
    '宠物殡葬', '宠物火化', '宠物安葬', '告别会', '纪念', '纪念册', '骨灰盒',
    '送最后一程', '最后的告别', '生命', '死亡', '临终'
}

# 无关内容关键词（需剔除的弹幕类型）
IRRELEVANT_KEYWORDS = {
    '冰冰', '王冰冰', '记者', '主持人', 'UP主', '视频', '画面', '镜头', '剪辑',
    '配乐', 'BGM', '弹幕', '评论', '点赞', '投币', '收藏', '关注', '转发',
    '直播', '更新', '催更', '鸽子', '拖更', 'UP', '主播', '颜值', '身材',
    '天气', '季节', '美食', '游戏', '追剧', '学习', '工作', '考试', '放假'
}

# ===================== 1. 安全保存文件（解决权限问题） =====================
def safe_save_excel(df, output_path):
    """安全保存Excel，自动处理文件占用、路径不存在问题"""
    # 创建文件夹（如果不存在）
    folder_path = os.path.dirname(output_path)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"📂 自动创建文件夹：{folder_path}")
    
    # 处理文件重名（添加时间戳避免覆盖）
    if os.path.exists(output_path):
        file_name, ext = os.path.splitext(output_path)
        timestamp = time.strftime("%Y%m%d_%H%M%S")
        output_path = f"{file_name}_{timestamp}{ext}"
        print(f"⚠️  原文件已存在，重命名为：{os.path.basename(output_path)}")
    
    # 3次重试保存（解决文件占用）
    for i in range(3):
        try:
            df.to_excel(output_path, index=False, engine='openpyxl')
            return output_path  # 保存成功，返回最终路径
        except PermissionError:
            if i == 2:
                print(f"❌ 保存失败：目标文件被占用，请关闭相关Excel后重试")
                return None
            print(f"⚠️ 文件被占用，{3-i}秒后重试...")
            time.sleep(1)
        except Exception as e:
            print(f"❌ 保存失败：{str(e)}")
            return None

# ===================== 2. 数据读取 =====================
def load_data(file_path):
    """读取原始弹幕数据"""
    try:
        df = pd.read_excel(file_path)
        print(f"✅ 成功读取数据：{len(df)}条弹幕，列名：{df.columns.tolist()}")
        return df
    except Exception as e:
        print(f"❌ 读取失败：{str(e)}")
        print("请检查：1.文件路径是否正确 2.文件是否被打开 3.是否安装openpyxl（pip install openpyxl）")
        return None

# ===================== 3. 主题相关性判断（核心过滤） =====================
def is_relevant(text):
    """判断弹幕是否与宠物殡葬相关"""
    if not text:
        return False
    
    # 正向匹配：包含相关关键词 → 保留
    for kw in PET_FUNERAL_KEYWORDS:
        if kw in text:
            return True
    
    # 反向排除：包含无关关键词 → 剔除
    for kw in IRRELEVANT_KEYWORDS:
        if kw in text:
            return False
    
    # 情感+场景匹配：无关键词但符合主题 → 保留
    emotion_words = {'泪目', '哭了', '难过', '心疼', '感动', '不舍', '怀念'}
    pet_leave_words = {'猫', '狗', '宠物', '毛孩子', '离开', '走了', '逝去', '告别'}
    if any(w in text for w in emotion_words) and any(w in text for w in pet_leave_words):
        return True
    
    return False

# ===================== 4. 弹幕文本清洗 =====================
def clean_text(text):
    """清洗文本：去除干扰字符、标准化格式"""
    if pd.isna(text) or text == '':
        return ''
    
    # 基础清理
    text = str(text).strip()
    text = re.sub(r'[\n\r\t]', '', text)  # 去除换行/制表符
    text = re.sub(r'\s+', ' ', text)      # 多个空格→单个空格
    
    # 过滤特殊符号（保留中文、英文、数字、常见标点）
    text = re.sub(r'[^\u4e00-\u9fa5a-zA-Z0-9，。！？；：""''（）【】、·…—\s]', '', text)
    
    # 去除无意义文本（长度<2且非情感词）
    meaningless = {'的', '了', '是', '在', '和', '啊', '呀', '呢', '哦', '吧'}
    if len(text) < 2 or (len(text) == 1 and text in meaningless):
        return ''
    
    # 简化重复字符（如"呜呜呜呜"→"呜呜"）
    text = re.sub(r'(.)\1{3,}', r'\1\1', text)
    
    return text

# ===================== 5. 完整处理流程 =====================
def process_data(df_original, output_path):
    """清洗+过滤+保存完整流程"""
    # 步骤1：基础文本清洗
    df_original['清洗后内容'] = df_original['内容'].apply(clean_text)
    df_cleaned = df_original[df_original['清洗后内容'] != ''].copy()  # 剔除无效文本
    
    # 步骤2：主题过滤（核心）
    print(f"\n🔍 正在过滤非宠物殡葬相关弹幕...")
    df_cleaned['是否相关'] = df_cleaned['清洗后内容'].apply(is_relevant)
    df_final = df_cleaned[df_cleaned['是否相关'] == True].copy().reset_index(drop=True)
    df_irrelevant = df_cleaned[df_cleaned['是否相关'] == False].copy()
    
    # 步骤3：添加辅助列
    df_final['弹幕序号'] = range(1, len(df_final) + 1)
    df_final['文本长度'] = df_final['清洗后内容'].apply(len)
    
    # 步骤4：整理输出格式
    df_output = df_final[['弹幕序号', '内容', '清洗后内容', '文本长度']].copy()
    df_output.rename(columns={'内容': '原始内容'}, inplace=True)
    
    # 步骤5：安全保存
    save_path = safe_save_excel(df_output, output_path)
    
    # 输出统计
    print(f"\n📊 过滤统计：")
    print(f"   • 清洗后有效文本：{len(df_cleaned)} 条")
    print(f"   • 剔除无关弹幕：{len(df_irrelevant)} 条")
    print(f"   • 最终保留相关弹幕：{len(df_final)} 条")
    
    return df_output, df_irrelevant, save_path

# ===================== 6. 生成清洗报告 =====================
def generate_report(df_original, df_final, df_irrelevant, save_path):
    """生成详细清洗报告"""
    print("\n" + "="*60)
    print("          宠物殡葬B站弹幕数据清洗报告（最终版）")
    print("="*60)
    
    # 基础统计
    total_original = len(df_original)
    total_final = len(df_final)
    filtered_invalid = total_original - len(df_original[df_original['清洗后内容'] != ''])
    filtered_irrelevant = len(df_irrelevant)
    
    print(f"📈 数据规模：")
    print(f"   • 原始数据：{total_original} 条")
    print(f"   • 剔除无效文本：{filtered_invalid} 条（格式错误/无意义）")
    print(f"   • 剔除无关主题：{filtered_irrelevant} 条（非宠物殡葬）")
    print(f"   • 最终有效数据：{total_final} 条")
    print(f"   • 核心数据保留率：{round(total_final/total_original*100, 2)}%")
    
    # 文本特征
    if total_final > 0:
        avg_len = round(df_final['文本长度'].mean(), 2)
        print(f"\n📝 文本特征：")
        print(f"   • 平均长度：{avg_len} 字符")
        print(f"   • 最长弹幕：{df_final['文本长度'].max()} 字符")
        print(f"   • 最短弹幕：{df_final['文本长度'].min()} 字符")
    
    # 无关弹幕示例（便于调整关键词）
    print(f"\n🚫 剔除的无关弹幕示例（前5条）：")
    for idx, row in df_irrelevant.head(5).iterrows():
        print(f"   • {row['内容']}")
    
    # 最终有效数据示例
    print(f"\n✅ 保留的相关弹幕示例（前5条）：")
    for idx, row in df_final.head(5).iterrows():
        print(f"   • 原始：{row['原始内容']}")
        print(f"   • 清洗：{row['清洗后内容']}")
    
    if save_path:
        print(f"\n📁 输出文件路径：{save_path}")
    print("="*60)

# ===================== 主函数（直接运行） =====================
def main():
    # ---------------------- 请修改这里的路径 ----------------------
    INPUT_FILE = r"D:\CITYU COURSES\5507\homework\弹幕\BV17JRKYdEEs弹幕数据.xlsx"
    OUTPUT_FILE = r"D:\CITYU COURSES\5507\homework\弹幕\BV17JRKYdEEs弹幕数据_最终清洗版.xlsx"
    # -------------------------------------------------------------
    
    # 执行流程
    df_original = load_data(INPUT_FILE)
    if df_original is None:
        return
    
    df_final, df_irrelevant, save_path = process_data(df_original, OUTPUT_FILE)
    generate_report(df_original, df_final, df_irrelevant, save_path)
    print("\n🎉 清洗完成！可直接使用输出文件进行情感分析～")

# 运行主函数
if __name__ == "__main__":
    main()
    

✅ 成功读取数据：872条弹幕，列名：['内容']

🔍 正在过滤非宠物殡葬相关弹幕...

📊 过滤统计：
   • 清洗后有效文本：851 条
   • 剔除无关弹幕：657 条
   • 最终保留相关弹幕：194 条

          宠物殡葬B站弹幕数据清洗报告（最终版）
📈 数据规模：
   • 原始数据：872 条
   • 剔除无效文本：21 条（格式错误/无意义）
   • 剔除无关主题：657 条（非宠物殡葬）
   • 最终有效数据：194 条
   • 核心数据保留率：22.25%

📝 文本特征：
   • 平均长度：12.4 字符
   • 最长弹幕：81 字符
   • 最短弹幕：2 字符

🚫 剔除的无关弹幕示例（前5条）：
   • “直行或是转弯，我们最终都会相见”
   • 人可以干干净净的走，它们也可以
   • 画面色彩富士味儿很足啊
   • 记者冰上线
   • “我们最终都会相见”

✅ 保留的相关弹幕示例（前5条）：
   • 原始：别说参与了就这么看都泪目
   • 清洗：别说参与了就这么看都泪目
   • 原始：乌龟：我的续航非常久
   • 清洗：乌龟：我的续航非常久
   • 原始：葬礼的意义 更多的是为了活着的人。一场庄重而温暖的仪式告别 让人心结 哀伤有所寄托。愿悲伤远离我们。
   • 清洗：葬礼的意义 更多的是为了活着的人。一场庄重而温暖的仪式告别 让人心结 哀伤有所寄托。愿悲伤远离我们。
   • 原始：晚安，猫猫～
   • 清洗：晚安，猫猫
   • 原始：意义或许是，让宠物的灵魂不会无家可归吧（）
   • 清洗：意义或许是，让宠物的灵魂不会无家可归吧（）

📁 输出文件路径：D:\CITYU COURSES\5507\homework\弹幕\BV17JRKYdEEs弹幕数据_最终清洗版.xlsx

🎉 清洗完成！可直接使用输出文件进行情感分析～


In [1]:
# 宠物殡葬B站弹幕清洗代码（修复KeyError，适配BV1AU4y1u79z）
import pandas as pd
import re
import os
import time
import warnings
warnings.filterwarnings('ignore')

# ===================== 1. 核心配置：主题关键词 =====================
RELEVANT_KEYWORDS = {
    # 宠物类型
    '猫', '喵', '猫咪', '小猫', '猫主子', '狗', '汪', '狗狗', '小狗', '宠物',
    '毛孩子', '主子', '小家伙', '小动物', '龟', '乌龟', '仓鼠', '兔子', '龙猫',
    # 殡葬相关
    '殡葬', '葬礼', '安葬', '火化', '告别', '送行', '安息', '长眠', '离世', '去世',
    '逝去', '离开', '走了', '往生', '后事', '告别仪式', '骨灰', '墓碑',
    # 核心情感
    '泪目', '哭了', '难过', '心疼', '感动', '温暖', '治愈', '安心', '不舍',
    '怀念', '思念', '尊重', '体面', '好好告别',
    # 场景相关
    '宠物殡葬', '宠物火化', '宠物安葬', '送最后一程', '最后的告别'
}

IRRELEVANT_KEYWORDS = {
    'UP主', '视频', '画面', '剪辑', '配乐', 'BGM', '弹幕', '评论',
    '点赞', '投币', '收藏', '关注', '直播', '更新', '催更', '主播',
    '天气', '季节', '美食', '游戏', '追剧', '学习', '工作', '考试'
}

# ===================== 2. 工具函数：自动识别弹幕列名 =====================
def get_comment_column(df):
    """自动识别弹幕内容列（应对不同列名格式）"""
    # 常见弹幕列名列表
    possible_columns = ['内容', '弹幕内容', 'comment', 'danmu', '弹幕', 'text']
    
    # 匹配列名（不区分大小写）
    for col in df.columns:
        col_lower = str(col).lower()
        for possible in possible_columns:
            if possible in str(col) or possible.lower() in col_lower:
                print(f"✅ 自动识别弹幕列：{col}")
                return col
    
    # 若未匹配到，提示用户选择
    print(f"\n⚠️  未自动识别弹幕列，当前列名：{df.columns.tolist()}")
    while True:
        selected_col = input("请输入弹幕内容所在列名（直接复制列名）：")
        if selected_col in df.columns:
            return selected_col
        print(f"❌ 列名{selected_col}不存在，请重新输入！")

# ===================== 3. 工具函数：安全保存文件 =====================
def safe_save_excel(df, output_path):
    folder = os.path.dirname(output_path)
    if not os.path.exists(folder):
        os.makedirs(folder)
    
    if os.path.exists(output_path):
        name, ext = os.path.splitext(output_path)
        timestamp = time.strftime("%Y%m%d_%H%M")
        output_path = f"{name}_{timestamp}{ext}"
    
    for i in range(3):
        try:
            df.to_excel(output_path, index=False, engine='openpyxl')
            return output_path
        except PermissionError:
            if i == 2:
                print(f"❌ 保存失败：文件被占用，请关闭Excel后重试")
                return None
            time.sleep(1)
        except Exception as e:
            print(f"❌ 保存失败：{str(e)}")
            return None

# ===================== 4. 核心清洗函数 =====================
def clean_bullet_text(text):
    if pd.isna(text) or text == '':
        return ''
    
    # 基础格式清理
    text = str(text).strip()
    text = re.sub(r'[\n\r\t]', '', text)
    text = re.sub(r'\s+', ' ', text)
    
    # 过滤特殊符号
    text = re.sub(r'[^\u4e00-\u9fa5a-zA-Z0-9，。！？；：""''（）【】、·…—\s]', '', text)
    
    # 去除无意义文本
    meaningless = {'的', '了', '是', '在', '和', '啊', '呀', '呢', '哦', '吧'}
    if len(text) < 2 or (len(text) == 1 and text in meaningless):
        return ''
    
    # 简化重复字符
    text = re.sub(r'(.)\1{3,}', r'\1\1', text)
    
    return text

# ===================== 5. 主题过滤函数 =====================
def is_pet_funeral_related(text):
    if not text:
        return False
    
    # 正向匹配相关关键词
    for kw in RELEVANT_KEYWORDS:
        if kw in text:
            return True
    
    # 反向排除无关关键词
    for kw in IRRELEVANT_KEYWORDS:
        if kw in text:
            return False
    
    # 情感+场景补充匹配
    emotion_words = {'泪目', '哭了', '难过', '心疼', '感动', '不舍', '怀念'}
    pet_leave_words = {'猫', '狗', '宠物', '毛孩子', '离开', '走了', '逝去', '告别'}
    if any(w in text for w in emotion_words) and any(w in text for w in pet_leave_words):
        return True
    
    return False

# ===================== 6. 完整清洗流程（自动适配列名） =====================
def full_clean_process(input_path, output_path):
    # 步骤1：读取数据并识别弹幕列
    try:
        df_original = pd.read_excel(input_path)
        print(f"✅ 成功读取数据：{len(df_original)}条记录，列名：{df_original.columns.tolist()}")
    except Exception as e:
        print(f"❌ 读取失败：{str(e)}")
        print("请检查路径或安装openpyxl（pip install openpyxl）")
        return None, None
    
    # 自动获取弹幕列名
    comment_col = get_comment_column(df_original)
    
    # 步骤2：文本清洗
    print(f"\n🔧 正在清洗{comment_col}列...")
    df_original['清洗后内容'] = df_original[comment_col].apply(clean_bullet_text)
    df_cleaned = df_original[df_original['清洗后内容'] != ''].copy()
    print(f"   清洗完成：剔除无效文本{len(df_original)-len(df_cleaned)}条，剩余{len(df_cleaned)}条")
    
    # 步骤3：主题过滤
    print(f"\n🎯 过滤非宠物殡葬相关弹幕...")
    df_cleaned['是否相关'] = df_cleaned['清洗后内容'].apply(is_pet_funeral_related)
    df_final = df_cleaned[df_cleaned['是否相关'] == True].copy().reset_index(drop=True)
    print(f"   过滤完成：剔除无关弹幕{len(df_cleaned)-len(df_final)}条，保留{len(df_final)}条相关弹幕")
    
    # 步骤4：整理输出格式
    df_final['弹幕序号'] = range(1, len(df_final)+1)
    df_final['文本长度'] = df_final['清洗后内容'].apply(len)
    
    # 保留原始内容列（重命名为统一的“原始内容”）
    df_output = df_final.rename(columns={comment_col: '原始内容'})
    df_output = df_output[['弹幕序号', '原始内容', '清洗后内容', '文本长度']].copy()
    
    # 步骤5：安全保存
    save_path = safe_save_excel(df_output, output_path)
    
    # 步骤6：输出报告
    print(f"\n" + "="*50)
    print("          清洗报告（BV1AU4y1u79z）")
    print("="*50)
    print(f"📊 数据统计：")
    print(f"   • 原始数据：{len(df_original)}条")
    print(f"   • 最终相关数据：{len(df_final)}条")
    print(f"   • 保留率：{round(len(df_final)/len(df_original)*100, 2)}%")
    if len(df_final) > 0:
        print(f"📝 文本特征：")
        print(f"   • 平均长度：{round(df_final['文本长度'].mean(), 2)}字符")
    print("="*50)
    
    return df_output, save_path

# ===================== 主执行函数 =====================
if __name__ == "__main__":
    # ---------------------- 本地路径配置 ----------------------
    INPUT_FILE_PATH = r"D:\CITYU COURSES\5507\homework\弹幕\BV1AU4y1u79z弹幕数据.xlsx"
    OUTPUT_FILE_PATH = r"D:\CITYU COURSES\5507\homework\弹幕\BV1AU4y1u79z弹幕数据_清洗完成版.xlsx"
    # ----------------------------------------------------------
    
    # 执行清洗
    cleaned_df, save_path = full_clean_process(INPUT_FILE_PATH, OUTPUT_FILE_PATH)
    if cleaned_df is not None and save_path:
        print(f"\n🎉 清洗完成！文件路径：{save_path}")
        print("   可直接用于宠物殡葬情感分析～")

✅ 成功读取数据：507条记录，列名：['弹幕内容']
✅ 自动识别弹幕列：弹幕内容

🔧 正在清洗弹幕内容列...
   清洗完成：剔除无效文本12条，剩余495条

🎯 过滤非宠物殡葬相关弹幕...
   过滤完成：剔除无关弹幕317条，保留178条相关弹幕

          清洗报告（BV1AU4y1u79z）
📊 数据统计：
   • 原始数据：507条
   • 最终相关数据：178条
   • 保留率：35.11%
📝 文本特征：
   • 平均长度：19.97字符

🎉 清洗完成！文件路径：D:\CITYU COURSES\5507\homework\弹幕\BV1AU4y1u79z弹幕数据_清洗完成版.xlsx
   可直接用于宠物殡葬情感分析～


In [1]:
# 宠物殡葬B站弹幕清洗代码（适配BV1iJ4m1G7kJ文件）
import pandas as pd
import re
import os
import time
import warnings
warnings.filterwarnings('ignore')

# ===================== 1. 核心配置：宠物殡葬主题关键词 =====================
# 相关关键词（覆盖宠物类型、殡葬场景、情感表达，确保主题精准）
RELEVANT_KEYWORDS = {
    # 宠物类型
    '猫', '喵', '猫咪', '小猫', '猫主子', '狗', '汪', '狗狗', '小狗', '宠物',
    '毛孩子', '主子', '小家伙', '小动物', '龟', '乌龟', '仓鼠', '兔子', '龙猫',
    '鹦鹉', '鸟', '鱼', '荷兰猪', '豚鼠',
    # 殡葬相关
    '殡葬', '葬礼', '安葬', '火化', '告别', '送行', '安息', '长眠', '离世', '去世',
    '逝去', '离开', '走了', '往生', '后事', '告别仪式', '骨灰', '墓碑', '下葬',
    # 核心情感
    '泪目', '哭了', '难过', '心疼', '感动', '温暖', '治愈', '安心', '不舍',
    '怀念', '思念', '想念', '尊重', '体面', '尊严', '好好告别', '心疼',
    # 场景相关
    '宠物殡葬', '宠物火化', '宠物安葬', '送最后一程', '最后的告别', '生命终点', '临终关怀'
}

# 无关关键词（剔除视频评价、闲聊等非主题内容）
IRRELEVANT_KEYWORDS = {
    'UP主', '视频', '画面', '剪辑', '配乐', 'BGM', '弹幕', '评论', '点赞',
    '投币', '收藏', '关注', '转发', '直播', '更新', '催更', '拖更', '鸽子',
    '主播', '颜值', '身材', '明星', '天气', '季节', '美食', '游戏', '追剧',
    '学习', '工作', '考试', '放假', '开学', '电影', '电视剧', '综艺'
}

# ===================== 2. 关键工具：自动识别弹幕列名（解决KeyError） =====================
def auto_detect_comment_column(df):
    """
    自动识别弹幕内容列，适配不同文件格式
    支持列名：内容、弹幕内容、comment、danmu、弹幕、text等
    """
    # 常见弹幕列名库（含中英文、大小写适配）
    target_patterns = ['内容', '弹幕', 'comment', 'danmu', 'text', '留言']
    
    # 遍历列名匹配
    for col in df.columns:
        col_lower = str(col).strip().lower()
        for pattern in target_patterns:
            if pattern in str(col) or pattern.lower() in col_lower:
                print(f"✅ 自动识别弹幕列：【{col}】")
                return col
    
    # 手动选择（若自动识别失败）
    print(f"\n⚠️  未自动匹配弹幕列，当前文件列名：")
    for i, col in enumerate(df.columns, 1):
        print(f"   {i}. {col}")
    
    while True:
        try:
            choice = int(input("请输入弹幕内容列对应的序号（如1）："))
            selected_col = df.columns[choice-1]
            print(f"✅ 您选择的弹幕列：【{selected_col}】")
            return selected_col
        except (ValueError, IndexError):
            print(f"❌ 输入无效，请重新输入正确序号！")

# ===================== 3. 安全保存：避免权限错误与文件覆盖 =====================
def safe_save_excel(df, output_path):
    """
    自动处理：
    1. 创建不存在的文件夹
    2. 重命名已存在的文件（加时间戳）
    3. 重试文件占用问题
    """
    # 1. 创建文件夹
    folder_path = os.path.dirname(output_path)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"📂 自动创建文件夹：{folder_path}")
    
    # 2. 处理文件重名
    if os.path.exists(output_path):
        file_name, ext = os.path.splitext(output_path)
        timestamp = time.strftime("%Y%m%d_%H%M%S")  # 精确到秒，避免重复
        output_path = f"{file_name}_{timestamp}{ext}"
        print(f"⚠️  目标文件已存在，重命名为：{os.path.basename(output_path)}")
    
    # 3. 3次重试保存（应对文件占用）
    for retry in range(3):
        try:
            df.to_excel(output_path, index=False, engine='openpyxl')
            return output_path
        except PermissionError:
            if retry == 2:
                print(f"❌ 保存失败：文件被占用，请关闭相关Excel窗口后重试！")
                return None
            print(f"⚠️ 文件被占用，{3-retry}秒后重试...")
            time.sleep(1)
        except Exception as e:
            print(f"❌ 保存失败：{str(e)}")
            return None

# ===================== 4. 文本清洗：去除干扰，保留情感信息 =====================
def clean_danmu_text(text):
    """
    单条弹幕清洗逻辑：
    1. 基础格式清理
    2. 特殊符号过滤
    3. 无意义文本剔除
    4. 重复字符简化
    """
    if pd.isna(text) or str(text).strip() == '':
        return ''
    
    # 1. 基础格式清理
    text = str(text).strip()
    text = re.sub(r'[\n\r\t]', '', text)  # 去除换行/制表符
    text = re.sub(r'\s+', ' ', text)      # 多个空格→单个空格
    
    # 2. 特殊符号过滤（保留中文、英文、数字、情感标点）
    text = re.sub(r'[^\u4e00-\u9fa5a-zA-Z0-9，。！？；：""''（）【】、·…—\s]', '', text)
    
    # 3. 剔除无意义文本（长度<2且非情感词）
    meaningless_chars = {'的', '了', '是', '在', '和', '与', '及', '等', '啊', '呀', '呢', '哦', '吧'}
    if len(text) < 2 or (len(text) == 1 and text in meaningless_chars):
        return ''
    
    # 4. 简化重复字符（如"呜呜呜呜"→"呜呜"，保留情感强度）
    text = re.sub(r'(.)\1{3,}', r'\1\1', text)
    
    return text

# ===================== 5. 主题过滤：仅保留宠物殡葬相关弹幕 =====================
def is_pet_funeral_related(text):
    """
    三层判断逻辑，确保主题精准：
    1. 正向匹配：含相关关键词→保留
    2. 反向排除：含无关关键词→剔除
    3. 补充匹配：情感+场景→保留
    """
    if not text:
        return False
    
    # 1. 正向匹配：含宠物殡葬相关关键词
    for kw in RELEVANT_KEYWORDS:
        if kw in text:
            return True
    
    # 2. 反向排除：含无关关键词
    for kw in IRRELEVANT_KEYWORDS:
        if kw in text:
            return False
    
    # 3. 补充匹配：无明确关键词但符合"情感+宠物/离别"场景
    emotion_words = {'泪目', '哭了', '难过', '心疼', '感动', '不舍', '怀念', '心疼'}
    pet_leave_words = {'猫', '狗', '宠物', '毛孩子', '离开', '走了', '逝去', '告别', '安息'}
    if any(word in text for word in emotion_words) and any(word in text for word in pet_leave_words):
        return True
    
    return False

# ===================== 6. 完整清洗流程：一键执行 =====================
def full_pet_funeral_clean(input_path, output_path):
    """
    完整流程：读取→识别列→清洗→过滤→保存→报告
    返回：清洗后DataFrame、保存路径
    """
    # 步骤1：读取原始数据
    try:
        df_original = pd.read_excel(input_path, engine='openpyxl')
        print(f"✅ 成功读取数据：共{len(df_original)}条记录")
    except Exception as e:
        print(f"❌ 数据读取失败：{str(e)}")
        print("请检查：1.文件路径是否正确 2.是否安装openpyxl（执行：pip install openpyxl）")
        return None, None
    
    # 步骤2：自动识别弹幕列
    comment_col = auto_detect_comment_column(df_original)
    
    # 步骤3：文本清洗
    print(f"\n🔧 开始文本清洗...")
    df_original['清洗后内容'] = df_original[comment_col].apply(clean_danmu_text)
    df_cleaned = df_original[df_original['清洗后内容'] != ''].copy()  # 剔除无效文本
    invalid_count = len(df_original) - len(df_cleaned)
    print(f"   清洗完成：剔除无效文本{invalid_count}条，剩余{len(df_cleaned)}条有效文本")
    
    # 步骤4：主题过滤（核心）
    print(f"\n🎯 开始宠物殡葬主题过滤...")
    df_cleaned['是否相关'] = df_cleaned['清洗后内容'].apply(is_pet_funeral_related)
    df_final = df_cleaned[df_cleaned['是否相关'] == True].copy().reset_index(drop=True)
    irrelevant_count = len(df_cleaned) - len(df_final)
    print(f"   过滤完成：剔除无关弹幕{irrelevant_count}条，保留{len(df_final)}条相关弹幕")
    
    # 步骤5：添加分析辅助列
    df_final['弹幕序号'] = range(1, len(df_final) + 1)  # 唯一序号
    df_final['文本长度'] = df_final['清洗后内容'].apply(len)  # 文本长度（情感强度参考）
    
    # 步骤6：整理输出格式（统一列名，便于后续分析）
    df_output = df_final.rename(columns={comment_col: '原始内容'})  # 统一原始列名
    df_output = df_output[['弹幕序号', '原始内容', '清洗后内容', '文本长度']].copy()
    
    # 步骤7：安全保存
    save_path = safe_save_excel(df_output, output_path)
    if not save_path:
        return None, None
    
    # 步骤8：输出清洗报告
    print(f"\n" + "="*60)
    print("          宠物殡葬弹幕清洗报告（BV1iJ4m1G7kJ）")
    print("="*60)
    print(f"📊 数据统计：")
    print(f"   • 原始数据总量：{len(df_original)}条")
    print(f"   • 最终相关数据：{len(df_final)}条")
    print(f"   • 数据保留率：{round(len(df_final)/len(df_original)*100, 2)}%")
    if len(df_final) > 0:
        print(f"📝 文本特征：")
        print(f"   • 平均文本长度：{round(df_final['文本长度'].mean(), 2)}字符")
        print(f"   • 最长弹幕长度：{df_final['文本长度'].max()}字符")
        print(f"   • 最短弹幕长度：{df_final['文本长度'].min()}字符")
    print(f"\n📁 输出文件路径：{save_path}")
    print("="*60)
    
    return df_output, save_path

# ===================== 主执行入口 =====================
if __name__ == "__main__":
    # ---------------------- 本地路径配置（请确认是否与您的文件一致） ----------------------
    INPUT_FILE = r"D:\CITYU COURSES\5507\homework\弹幕\BV1iJ4m1G7kJ弹幕数据.xlsx"
    OUTPUT_FILE = r"D:\CITYU COURSES\5507\homework\弹幕\BV1iJ4m1G7kJ弹幕数据_清洗完成版.xlsx"
    # -------------------------------------------------------------------------------------
    
    # 执行完整清洗
    cleaned_data, saved_path = full_pet_funeral_clean(INPUT_FILE, OUTPUT_FILE)
    if cleaned_data is not None and saved_path:
        print(f"\n🎉 清洗任务完成！可直接使用【{os.path.basename(saved_path)}】制作情感分析图～")

✅ 成功读取数据：共793条记录
✅ 自动识别弹幕列：【弹幕内容】

🔧 开始文本清洗...
   清洗完成：剔除无效文本12条，剩余781条有效文本

🎯 开始宠物殡葬主题过滤...
   过滤完成：剔除无关弹幕593条，保留188条相关弹幕

          宠物殡葬弹幕清洗报告（BV1iJ4m1G7kJ）
📊 数据统计：
   • 原始数据总量：793条
   • 最终相关数据：188条
   • 数据保留率：23.71%
📝 文本特征：
   • 平均文本长度：18.71字符
   • 最长弹幕长度：64字符
   • 最短弹幕长度：2字符

📁 输出文件路径：D:\CITYU COURSES\5507\homework\弹幕\BV1iJ4m1G7kJ弹幕数据_清洗完成版.xlsx

🎉 清洗任务完成！可直接使用【BV1iJ4m1G7kJ弹幕数据_清洗完成版.xlsx】制作情感分析图～


In [1]:
# 宠物殡葬B站弹幕清洗代码（适配BV16Z4y1K7PM文件）
import pandas as pd
import re
import os
import time
import warnings
warnings.filterwarnings('ignore')

# ===================== 1. 核心配置：宠物殡葬主题关键词库 =====================
# 相关关键词（覆盖宠物类型、殡葬场景、情感表达，确保主题精准）
RELEVANT_KEYWORDS = {
    # 宠物类型（扩充常见宠物）
    '猫', '喵', '猫咪', '小猫', '猫主子', '狗', '汪', '狗狗', '小狗', '宠物',
    '毛孩子', '主子', '小家伙', '小动物', '龟', '乌龟', '仓鼠', '兔子', '龙猫',
    '鹦鹉', '鸟', '鱼', '荷兰猪', '豚鼠', '仓鼠', '蜜袋鼯', '花枝鼠',
    # 殡葬核心场景
    '殡葬', '葬礼', '安葬', '火化', '告别', '送行', '安息', '长眠', '离世', '去世',
    '逝去', '离开', '走了', '往生', '后事', '告别仪式', '骨灰', '墓碑', '下葬',
    '宠物殡葬', '宠物火化', '宠物安葬', '宠物葬礼',
    # 情感表达（适配宠物殡葬情感倾向）
    '泪目', '哭了', '难过', '心疼', '感动', '温暖', '治愈', '安心', '不舍',
    '怀念', '思念', '想念', '尊重', '体面', '尊严', '好好告别', '心疼', '心碎',
    # 相关场景补充
    '送最后一程', '最后的告别', '生命终点', '临终关怀', '纪念', '缅怀', '悼念'
}

# 无关关键词（剔除视频评价、闲聊等非主题内容）
IRRELEVANT_KEYWORDS = {
    'UP主', '视频', '画面', '剪辑', '配乐', 'BGM', '弹幕', '评论', '点赞',
    '投币', '收藏', '关注', '转发', '直播', '更新', '催更', '拖更', '鸽子',
    '主播', '颜值', '身材', '明星', '天气', '季节', '美食', '游戏', '追剧',
    '学习', '工作', '考试', '放假', '开学', '电影', '电视剧', '综艺', '动漫',
    '打卡', '签到', '沙发', '前排', '围观', '路过', '飘过'
}

# ===================== 2. 关键工具：自动识别弹幕列名（解决KeyError） =====================
def auto_detect_comment_col(df):
    """
    自动识别弹幕内容列，适配不同文件格式：
    - 支持列名：内容、弹幕内容、comment、danmu、弹幕、text等
    - 自动容错，识别失败时引导手动选择
    """
    # 常见弹幕列名匹配规则
    target_col_patterns = ['内容', '弹幕', 'comment', 'danmu', 'text', '留言', '信息']
    
    # 遍历列名匹配（不区分大小写、空格）
    for col in df.columns:
        col_clean = str(col).strip().lower()
        for pattern in target_col_patterns:
            if pattern in str(col) or pattern.lower() in col_clean:
                print(f"✅ 自动识别弹幕列：【{col}】")
                return col
    
    # 手动选择流程（自动识别失败时）
    print(f"\n⚠️  未自动匹配弹幕列，当前文件所有列名：")
    for idx, col in enumerate(df.columns, 1):
        print(f"   {idx}. {col}")
    
    # 输入验证，确保选择有效列
    while True:
        try:
            user_choice = int(input("请输入弹幕内容所在列的序号（如1/2）："))
            selected_col = df.columns[user_choice - 1]
            print(f"✅ 确认选择弹幕列：【{selected_col}】")
            return selected_col
        except (ValueError, IndexError):
            print(f"❌ 输入无效！请输入列名对应的数字序号（如1对应第一列）")

# ===================== 3. 安全保存：避免权限错误与文件覆盖 =====================
def safe_save_excel(df, output_path):
    """
    安全保存逻辑：
    1. 自动创建目标文件夹（若不存在）
    2. 重命名已存在文件（添加时间戳，避免覆盖）
    3. 3次重试机制（解决文件被占用问题）
    """
    # 1. 创建目标文件夹
    folder_dir = os.path.dirname(output_path)
    if not os.path.exists(folder_dir):
        os.makedirs(folder_dir)
        print(f"📂 自动创建目标文件夹：{folder_dir}")
    
    # 2. 处理文件重名（添加精确时间戳）
    if os.path.exists(output_path):
        file_name, file_ext = os.path.splitext(output_path)
        timestamp = time.strftime("%Y%m%d_%H%M%S")  # 精确到秒，避免重复
        output_path = f"{file_name}_{timestamp}{file_ext}"
        print(f"⚠️  目标文件已存在，自动重命名为：{os.path.basename(output_path)}")
    
    # 3. 重试保存（应对文件占用）
    for retry_cnt in range(3):
        try:
            df.to_excel(output_path, index=False, engine='openpyxl')
            return output_path
        except PermissionError:
            if retry_cnt == 2:
                print(f"❌ 保存失败：文件被占用，请关闭相关Excel窗口后重试！")
                return None
            print(f"⚠️ 文件被占用，{3 - retry_cnt}秒后重试...")
            time.sleep(1)
        except Exception as e:
            print(f"❌ 保存失败：{str(e)}")
            return None

# ===================== 4. 文本清洗：去除干扰，保留情感核心信息 =====================
def clean_danmu_text(text):
    """
    单条弹幕清洗逻辑（适配情感分析需求）：
    1. 基础格式清理 → 2. 特殊符号过滤 → 3. 无意义文本剔除 → 4. 重复字符简化
    """
    # 处理空值/非字符串类型
    if pd.isna(text) or not isinstance(text, (str, int, float)):
        return ''
    
    # 1. 基础格式清理
    text = str(text).strip()
    text = re.sub(r'[\n\r\t]', '', text)  # 去除换行、回车、制表符
    text = re.sub(r'\s+', ' ', text)      # 多个空格→单个空格（保留情感断句）
    
    # 2. 特殊符号过滤（仅保留中文、英文、数字、情感相关标点）
    text = re.sub(r'[^\u4e00-\u9fa5a-zA-Z0-9，。！？；：""''（）【】、·…—\s]', '', text)
    
    # 3. 剔除无意义文本（避免干扰情感分析）
    meaningless_filter = {'的', '了', '是', '在', '和', '与', '及', '等', '啊', '呀', '呢', '哦', '吧', '呀'}
    if len(text) < 2 or (len(text) == 1 and text in meaningless_filter):
        return ''
    
    # 4. 简化重复字符（保留情感强度，避免冗余）
    text = re.sub(r'(.)\1{3,}', r'\1\1', text)  # 如"呜呜呜呜"→"呜呜"，"难过难过难过"→"难过难过"
    
    return text

# ===================== 5. 主题过滤：仅保留宠物殡葬相关弹幕 =====================
def is_pet_funeral_related(text):
    """
    三层主题判断逻辑（确保精准度）：
    1. 正向匹配：含相关关键词 → 保留
    2. 反向排除：含无关关键词 → 剔除
    3. 补充匹配：情感+宠物/离别场景 → 保留
    """
    if not text:
        return False
    
    # 1. 正向匹配：包含宠物殡葬相关关键词
    for relevant_kw in RELEVANT_KEYWORDS:
        if relevant_kw in text:
            return True
    
    # 2. 反向排除：包含无关关键词（优先剔除，减少误判）
    for irrelevant_kw in IRRELEVANT_KEYWORDS:
        if irrelevant_kw in text:
            return False
    
    # 3. 补充匹配：无明确关键词但符合"情感+宠物/离别"场景（避免漏判）
    emotion_keywords = {'泪目', '哭了', '难过', '心疼', '感动', '不舍', '怀念', '心碎', '体面'}
    pet_leave_keywords = {'猫', '狗', '宠物', '毛孩子', '离开', '走了', '逝去', '告别', '安息'}
    if any(emotion_kw in text for emotion_kw in emotion_keywords) and \
       any(pet_leave_kw in text for pet_leave_kw in pet_leave_keywords):
        return True
    
    # 其他情况：判定为无关
    return False

# ===================== 6. 完整清洗流程：一键执行（读取→清洗→过滤→保存） =====================
def full_pet_funeral_clean(input_file_path, output_file_path):
    """
    完整清洗流程，返回清洗后数据与保存路径：
    Step1：读取原始数据 → Step2：识别弹幕列 → Step3：文本清洗 → Step4：主题过滤 → Step5：安全保存 → Step6：输出报告
    """
    # Step1：读取原始数据
    try:
        df_original = pd.read_excel(input_file_path, engine='openpyxl')
        print(f"✅ 成功读取原始数据：共{len(df_original)}条弹幕记录")
    except Exception as e:
        print(f"❌ 原始数据读取失败：{str(e)}")
        print("请检查：1.文件路径是否正确 2.是否安装openpyxl（执行命令：pip install openpyxl）")
        return None, None
    
    # Step2：自动识别弹幕列
    comment_col = auto_detect_comment_col(df_original)
    
    # Step3：文本清洗（批量处理）
    print(f"\n🔧 开始批量文本清洗...")
    df_original['清洗后内容'] = df_original[comment_col].apply(clean_danmu_text)
    # 剔除清洗后为空的无效数据
    df_cleaned_text = df_original[df_original['清洗后内容'] != ''].copy()
    invalid_text_count = len(df_original) - len(df_cleaned_text)
    print(f"   文本清洗完成：剔除无效文本{invalid_text_count}条，剩余{len(df_cleaned_text)}条有效文本")
    
    # Step4：主题过滤（核心步骤，聚焦宠物殡葬）
    print(f"\n🎯 开始宠物殡葬主题过滤...")
    df_cleaned_text['是否相关'] = df_cleaned_text['清洗后内容'].apply(is_pet_funeral_related)
    # 保留仅相关的数据
    df_final = df_cleaned_text[df_cleaned_text['是否相关'] == True].copy().reset_index(drop=True)
    irrelevant_count = len(df_cleaned_text) - len(df_final)
    print(f"   主题过滤完成：剔除无关弹幕{irrelevant_count}条，保留{len(df_final)}条宠物殡葬相关弹幕")
    
    # Step5：添加情感分析辅助列
    df_final['弹幕序号'] = range(1, len(df_final) + 1)  # 唯一标识序号
    df_final['文本长度'] = df_final['清洗后内容'].apply(len)  # 文本长度（辅助情感强度分析）
    
    # Step6：整理输出格式（统一列名，便于后续分析）
    df_output = df_final.rename(columns={comment_col: '原始内容'})  # 统一原始列名
    # 保留核心列：弹幕序号、原始内容、清洗后内容、文本长度
    df_output = df_output[['弹幕序号', '原始内容', '清洗后内容', '文本长度']].copy()
    
    # Step7：安全保存清洗后数据
    saved_file_path = safe_save_excel(df_output, output_file_path)
    if not saved_file_path:
        return None, None
    
    # Step8：输出清洗报告（关键统计信息）
    print(f"\n" + "="*60)
    print("          宠物殡葬弹幕清洗报告（BV16Z4y1K7PM）")
    print("="*60)
    print(f"📊 数据规模统计：")
    print(f"   • 原始数据总量：{len(df_original)}条")
    print(f"   • 最终相关数据：{len(df_final)}条")
    print(f"   • 数据保留率：{round(len(df_final)/len(df_original)*100, 2)}%")
    if len(df_final) > 0:
        print(f"📝 文本特征统计：")
        print(f"   • 平均文本长度：{round(df_final['文本长度'].mean(), 2)}字符")
        print(f"   • 最长弹幕长度：{df_final['文本长度'].max()}字符")
        print(f"   • 最短弹幕长度：{df_final['文本长度'].min()}字符")
    print(f"\n📁 清洗后文件路径：{saved_file_path}")
    print("="*60)
    
    return df_output, saved_file_path

# ===================== 主执行入口（直接运行） =====================
if __name__ == "__main__":
    # ---------------------- 本地路径配置（请确认与您的文件一致） ----------------------
    INPUT_FILE = r"D:\CITYU COURSES\5507\homework\弹幕\BV16Z4y1K7PM弹幕数据.xlsx"
    OUTPUT_FILE = r"D:\CITYU COURSES\5507\homework\弹幕\BV16Z4y1K7PM弹幕数据_清洗完成版.xlsx"
    # -------------------------------------------------------------------------------------
    
    # 执行完整清洗流程
    cleaned_data, saved_path = full_pet_funeral_clean(INPUT_FILE, OUTPUT_FILE)
    if cleaned_data is not None and saved_path:
        print(f"\n🎉 清洗任务完成！可直接使用【{os.path.basename(saved_path)}】制作宠物殡葬情感分析图～")

✅ 成功读取原始数据：共1200条弹幕记录
✅ 自动识别弹幕列：【弹幕内容】

🔧 开始批量文本清洗...
   文本清洗完成：剔除无效文本7条，剩余1193条有效文本

🎯 开始宠物殡葬主题过滤...
   主题过滤完成：剔除无关弹幕722条，保留471条宠物殡葬相关弹幕

          宠物殡葬弹幕清洗报告（BV16Z4y1K7PM）
📊 数据规模统计：
   • 原始数据总量：1200条
   • 最终相关数据：471条
   • 数据保留率：39.25%
📝 文本特征统计：
   • 平均文本长度：13.82字符
   • 最长弹幕长度：100字符
   • 最短弹幕长度：2字符

📁 清洗后文件路径：D:\CITYU COURSES\5507\homework\弹幕\BV16Z4y1K7PM弹幕数据_清洗完成版.xlsx

🎉 清洗任务完成！可直接使用【BV16Z4y1K7PM弹幕数据_清洗完成版.xlsx】制作宠物殡葬情感分析图～
